In [7]:
import pandas as pd
import math
import scipy.stats as st
import numpy as np


pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv("amazon_review.csv")

df = data.copy()

In [3]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00000,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0


In [4]:
def missing_values_analysis(df):
    na_columns_ = [col for col in df.columns if df[col].isnull().sum() > 0]
    n_miss = df[na_columns_].isnull().sum().sort_values(ascending=True)
    ratio_ = (df[na_columns_].isnull().sum() / df.shape[0] * 100).sort_values(ascending=True)
    missing_df = pd.concat([n_miss, np.round(ratio_, 2)], axis=1, keys=['Total Missing Values', 'Ratio'])
    missing_df = pd.DataFrame(missing_df)
    return missing_df

In [8]:
# Let's define the auxiliary functions that we will use in the study
def check_df(df, head=5):
    print("--------------------- Shape ---------------------")
    print(df.shape)

    print("---------------------- Types --------------------")
    print(df.dtypes)

    print("--------------------- Head ---------------------")
    print(df.head(head))

    print("--------------------- Missing Value Analysis ---------------------")
    print(missing_values_analysis(df))

    print("--------------------- Quantiles ---------------------")
    print(df.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

In [9]:
check_df(df)

--------------------- Shape ---------------------
(4915, 12)
---------------------- Types --------------------
reviewerID         object
asin               object
reviewerName       object
helpful            object
reviewText         object
overall           float64
summary            object
unixReviewTime      int64
reviewTime         object
day_diff            int64
helpful_yes         int64
total_vote          int64
dtype: object
--------------------- Head ---------------------
       reviewerID        asin  reviewerName helpful                                         reviewText  overall                                 summary  unixReviewTime  reviewTime  day_diff  helpful_yes  total_vote
0  A3SBTW3WS4IQSN  B007WTAJTO           NaN  [0, 0]                                         No issues.  4.00000                              Four Stars      1406073600  2014-07-23       138            0           0
1  A18K1ODH1I2MVB  B007WTAJTO          0mie  [0, 0]  Purchased this for my device, i

In [5]:
df.shape

(4915, 12)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      4915 non-null   object 
 1   asin            4915 non-null   object 
 2   reviewerName    4914 non-null   object 
 3   helpful         4915 non-null   object 
 4   reviewText      4914 non-null   object 
 5   overall         4915 non-null   float64
 6   summary         4915 non-null   object 
 7   unixReviewTime  4915 non-null   int64  
 8   reviewTime      4915 non-null   object 
 9   day_diff        4915 non-null   int64  
 10  helpful_yes     4915 non-null   int64  
 11  total_vote      4915 non-null   int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 460.9+ KB


In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
overall,4915.00000,4.58759,0.99685,1.00000,5.00000,5.00000,5.00000,5.00000
unixReviewTime,4915.00000,1379465001.66836,15818574.32275,1339200000.00000,1365897600.00000,1381276800.00000,1392163200.00000,1406073600.00000
day_diff,4915.00000,437.36704,209.43987,1.00000,281.00000,431.00000,601.00000,1064.00000
helpful_yes,4915.00000,1.31109,41.61916,0.00000,0.00000,0.00000,0.00000,1952.00000
total_vote,4915.00000,1.52146,44.12309,0.00000,0.00000,0.00000,0.00000,2020.00000


In [9]:
df['asin'].value_counts()

B007WTAJTO    4915
Name: asin, dtype: int64

# Task 1: Calculate the Average Rating based on current comments and compare it with the existing average rating

## Step 1: Calculate the average score of the product.

In [10]:
df['overall'].mean()

4.587589013224822

## Step 2: Calculate the weighted average score by date

In [11]:
df["reviewTime"].max()

'2014-12-07'

In [12]:
df["reviewTime"] = pd.to_datetime(df["reviewTime"], dayfirst=True)
current_date = pd.to_datetime("2014-12-07")

df["days"] = (current_date - df["reviewTime"]).dt.days

In [13]:
df.days.describe().T

count   4915.00000
mean     436.36704
std      209.43987
min        0.00000
25%      280.00000
50%      430.00000
75%      600.00000
max     1063.00000
Name: days, dtype: float64

In [14]:
q_1 = df["days"].quantile(0.25)

In [15]:
q_2 = df["days"].quantile(0.50)

In [16]:
q_3 = df["days"].quantile(0.75)

In [17]:
print(df.loc[(df["days"] <= q_1), "overall"].mean())

4.6957928802588995


In [27]:
print(df.loc[(df["days"] > q_1) & (df["day_diff"] <= q_2), "overall"].mean())

4.637335526315789


In [28]:
print(df.loc[(df["days"] > q_2) & (df["day_diff"] <= q_3), "overall"].mean())

4.571428571428571


In [29]:
print(df.loc[(df["days"] > q_3), "overall"].mean())

4.4462540716612375


## Step 3: Compare and interpret the average of each time period in weighted scoring.

# Task 2: Determine the 20 reviews to be displayed on the product detail page for the product.

## Step 1: Generate the helpful_no variable.

In [30]:
df["helpful_no"] = df["total_vote"] - df["helpful_yes"]

In [31]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,days,helpful_no
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00000,Four Stars,1406073600,2014-07-23,138,0,0,137,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,408,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0,714,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0,381,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0,512,0


In [32]:
df = df[["overall", "reviewTime", "day_diff", "helpful_yes", "helpful_no","total_vote"]]
df.head()

,overall,reviewTime,day_diff,helpful_yes,helpful_no,total_vote
0,4.00000,2014-07-23,138,0,0,0
1,5.00000,2013-10-25,409,0,0,0
2,4.00000,2012-12-23,715,0,0,0
3,5.00000,2013-11-21,382,0,0,0
4,5.00000,2013-07-13,513,0,0,0


## Step 2: Calculate score_pos_neg_diff, score_average_rating and wilson_lower_bound scores and add them to the data

In [33]:
def wilson_lower_bound(up, down, confidence=0.95):
    """
    Calculate Wilson Lower Bound Score
    - The lower limit of the confidence interval to be calculated for the Bernoulli parameter p is accepted as the WLB score.
    - The score to be calculated is used for product ranking.
    - Note:
    If the scores are between 1-5, 1-3 are marked as negative, 4-5 as positive and can be adapted to bernoulli. This brings with it some problems. For this reason, it is necessary to make a bayesian average rating.
    
    Parameters
    ----------
    up: int
        up count
    down: int
        down count
    confidence: float
        confidence
    Returns
    -------
    wilson score: float
    """
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

In [34]:
def score_up_down_diff(up, down):
    return up - down

In [35]:
def score_average_rating(up, down):
    if up + down == 0:
        return 0
    return up / (up + down)

In [36]:
df["score_pos_neg_diff"] = df.apply(lambda x: score_up_down_diff(x["helpful_yes"], x["helpful_no"]), axis=1)

df["score_average_rating"] = df.apply(lambda x: score_average_rating(x["helpful_yes"], x["helpful_no"]), axis=1)

df["wilson_lower_bound"] = df.apply(lambda x: wilson_lower_bound(x["helpful_yes"], x["helpful_no"]), axis=1)

In [37]:
df.head()

,overall,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,4.00000,2014-07-23,138,0,0,0,0,0.00000,0.00000
1,5.00000,2013-10-25,409,0,0,0,0,0.00000,0.00000
2,4.00000,2012-12-23,715,0,0,0,0,0.00000,0.00000
3,5.00000,2013-11-21,382,0,0,0,0,0.00000,0.00000
4,5.00000,2013-07-13,513,0,0,0,0,0.00000,0.00000


## Step 3: Identify the 20 Interpretations and Interpret the results.

In [39]:
df.sort_values("wilson_lower_bound", ascending=False)[:20]

,overall,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,5.00000,2013-01-05,702,1952,68,2020,1884,0.96634,0.95754
3449,5.00000,2012-09-26,803,1428,77,1505,1351,0.94884,0.93652
4212,1.00000,2013-05-08,579,1568,126,1694,1442,0.92562,0.91214
317,1.00000,2012-02-09,1033,422,73,495,349,0.85253,0.81858
4672,5.00000,2014-07-03,158,45,4,49,41,0.91837,0.80811
1835,5.00000,2014-02-28,283,60,8,68,52,0.88235,0.78465
3981,5.00000,2012-10-22,777,112,27,139,85,0.80576,0.73214
3807,3.00000,2013-02-27,649,22,3,25,19,0.88000,0.70044
4306,5.00000,2012-09-06,823,51,14,65,37,0.78462,0.67033
4596,1.00000,2012-09-22,807,82,27,109,55,0.75229,0.66359
